In [1]:
from owlready2 import *
from pathlib import Path, PureWindowsPath
from pyvis.network import Network

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
path_to_ontology = PureWindowsPath('.')
print(path_to_ontology)
onto_path.append(path_to_ontology)
default_world.set_backend(filename = "fma.sqlite3")
onto = get_ontology('http://sig.biostr.washington.edu/share/downloads/fma/release/latest/fma.owl').load()

.


In [3]:
onto.base_iri

'http://purl.org/sig/ont/fma.owl#'

In [4]:
fma = get_namespace('http://purl.org/sig/ont/fma/')
vagus = fma.fma5731
left_vagus = fma.fma6220
right_vagus = fma.fma6219
print(vagus.iri, left_vagus.label, right_vagus.synonym)

http://purl.org/sig/ont/fma/fma5731 ['Left vagus nerve'] ['Right vagus', 'Right vagus nerve tree']


In [5]:
# Use SPARQL to count number of owl:class'es in the ontology.
list(default_world.sparql("""
           SELECT (COUNT(?x) AS ?nb)
           { ?x a owl:Class . }
    """))

[[106738]]

In [6]:
# Find all sub-classes of the vagus
def getSubClasses(base):
    res = list(default_world.sparql("""SELECT ?x { ?x rdfs:subClassOf ?? . }""", [base]))
    return res

In [7]:
def printLabels(classes):
    for c in classes:
        print(c[0].label)

In [8]:
printLabels(getSubClasses(vagus))

['Right vagus nerve']
['Left vagus nerve']


In [9]:
def getBranches(root):
    branches = list(default_world.sparql("""
    SELECT ?x {
        ?x rdfs:subClassOf [a owl:Restriction ;
                            owl:onProperty <http://purl.org/sig/ont/fma/branch_of> ;
                            owl:someValuesFrom ??] .
    }
    """, [root]))
    return branches

def listBranches(root):
    branches = getBranches(root)
    printLabels(branches)

In [10]:
print('Vagus branches:')
listBranches(vagus)
print('\nLeft vagus branches:')
listBranches(left_vagus)
print('\nRight vagus branches:')
listBranches(right_vagus)

Vagus branches:
['Recurrent laryngeal nerve']
['Inferior cervical cardiac branch of vagus nerve']
['Superior cervical cardiac branch of vagus nerve']
['Superior laryngeal nerve']
['Branch of vagus nerve to carotid body']
['Pharyngeal branch of vagus nerve']
['Communicating branch of vagus nerve with glossopharyngeal nerve']
['Auricular branch of vagus nerve']
['Meningeal branch of vagus nerve']
['Posterior vagal trunk']
['Anterior vagal trunk']

Left vagus branches:
['Esophageal branch of posterior vagal trunk']
['Inferior cervical cardiac branch of left vagus nerve to superficial cardiac plexus']
['Pulmonary branch of left vagus nerve']
['Renal branch of left vagus']
['Pharyngeal branch of left vagus nerve to pharyngeal nerve plexus']
['Lingual branch of left vagus nerve']
['Vagal branch to left carotid body']
['Thoracic cardiac branch of left vagus nerve']
['Superior cervical cardiac branch of left vagus nerve']
['Communicating branch of left vagus nerve with left glossopharyngeal ne

In [11]:
def buildBranchGraph(root, g):
    branches = getBranches(root)
    for b in branches:
        node = b[0]
        g.add_node(node.iri, label=f"fma:{node.FMAID[0]}", title=node.label[0])
        g.add_edge(root.iri, node.iri)
        buildBranchGraph(node, g)
        
graph = Network(notebook=True, cdn_resources='remote')
graph.add_node(vagus.iri, label=vagus.label[0], title=vagus.iri)
graph.add_node(left_vagus.iri, label=left_vagus.label[0], title=left_vagus.iri)
graph.add_node(right_vagus.iri, label=right_vagus.label[0], title=right_vagus.iri)
graph.add_edge(vagus.iri, left_vagus.iri)
graph.add_edge(vagus.iri, right_vagus.iri)
buildBranchGraph(left_vagus, graph)
buildBranchGraph(right_vagus, graph)

In [12]:
graph.show('vagus_combined.html')

vagus_combined.html


In [13]:
vagus_graph = Network(notebook=True, cdn_resources='remote')
vagus_graph.add_node(vagus.iri, label=vagus.label[0], title=vagus.iri)
buildBranchGraph(vagus, vagus_graph)
vagus_graph.show('vagus.html')

vagus.html
